In [1]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta, date
import polars as pl
from collections import defaultdict, Counter
from typing import Tuple
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
import matplotlib.pyplot as plt
import numpy as np
import os
from common.utils import (
    draw_chart,
    filter_by_date_range,
)

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)

from typing import Any
import matplotlib.dates as mdates
from matplotlib.patches import Patch
from scipy.stats import fisher_exact

pl.Config.set_tbl_rows(-1)
import statistics

POJISTOVNA = "cpzp"
VAX_PERIOD_IN_DAYS = 30
START_DATE = date(2015, 1, 1)
END_DATE = date(2025, 1, 1)
PERIOD = 365


def is_injection(form: str) -> bool:
    return form in {"Injekční suspenze", "Injekční/infuzní roztok"}


def skip_person_for_novax(p) -> bool:
    return (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni is not None and p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or bool(p.vaccines)
        or not p.prescriptions
    )


def skip_person_for_vax(p) -> bool:
    return (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni is not None and p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or (not p.vaccines)
        or (not p.prescriptions)
    )


def collapse_injections(prescriptions):
    """Yield prescriptions but ignore additional injections within 14 days."""
    last_inj_date = date.min
    for pr in prescriptions:
        if is_injection(pr.lekova_forma):
            if abs((last_inj_date - pr.date).days) < 14:
                continue
            last_inj_date = pr.date
        yield pr


def safe_div(a: float, b: float) -> float:
    return np.nan if b == 0 else a / b


def pvalue_from_df(df: pl.DataFrame) -> float:
    """Fisher exact test on Polars df with rows očkovaní/neočkovaní, cols před/po"""
    table = [
        [
            int(df.filter(pl.col("group") == "očkovaní")["před"][0]),
            int(df.filter(pl.col("group") == "očkovaní")["po"][0]),
        ],
        [
            int(df.filter(pl.col("group") == "neočkovaní")["před"][0]),
            int(df.filter(pl.col("group") == "neočkovaní")["po"][0]),
        ],
    ]
    return fisher_exact(table, alternative="two-sided")[1]


def before_after_df(
    vax_map: dict[int, float], novax_map: dict[int, float], pivot: int = 0
) -> pl.DataFrame:
    def sums(m):
        before = sum(v for d, v in m.items() if d < pivot)
        after = sum(v for d, v in m.items() if d >= pivot)
        return before, after

    vb, va = sums(vax_map)
    nb, na = sums(novax_map)
    return pl.DataFrame(
        {
            "group": ["očkovaní", "neočkovaní"],
            "před": [float(vb), float(nb)],
            "po": [float(va), float(na)],
        }
    )

In [2]:
if POJISTOVNA == "both_companies":
    with open("./DATACON_data/cpzp_persons.pkl", "rb") as f:
        cpzp_persons: list[Person] = pickle.load(f)
    with open("./DATACON_data/ozp_persons.pkl", "rb") as f:
        ozp_persons: list[Person] = pickle.load(f)
    persons = cpzp_persons + ozp_persons
else:
    with open(f"./DATACON_data/{POJISTOVNA}_persons.pkl", "rb") as f:
        persons: list[Person] = pickle.load(f)

In [3]:
vax_start = date(2021, 5, 13)
rozhodne_datum_pro_neockovane = date(2021, 6, 13)
vax_end = date(2021, 7, 13)

START_DATE = date(2015, 1, 1)
END_DATE = date(2025, 1, 1)

In [4]:
novax_ppl = 0
novax_ppl_alive = 0
novax_ppl_not_changing_insurance = 0

for p in persons:
    if not p.vaccines:
        novax_ppl += 1

        if not p.died_at:
            novax_ppl_alive += 1

            if p.zahajeni_pojisteni < START_DATE and p.ukonceni_pojisteni > END_DATE:
                novax_ppl_not_changing_insurance += 1


print(novax_ppl)
print(novax_ppl_alive)
print(novax_ppl_not_changing_insurance)

804808
727068
345001


In [22]:
ppl = 0
vax_ppl = 0
vax_ppl_alive = 0
vax_ppl_not_changing_insurance = 0
vax_ppl_between_30_and_50 = 0
vax_ppl_between_30_and_50_in_vax_period = 0

prescriptions_of_ppl_in_period = 0
prescriptions_of_ppl_before_vax = 0
prescriptions_of_ppl_after_vax = 0
prescriptions_of_ppl_at_vax = 0

first_prescriptions_of_ppl_in_period = 0
first_prescriptions_of_ppl_before_vax = 0
first_prescriptions_of_ppl_after_vax = 0
first_prescriptions_of_ppl_at_vax = 0

ids = []

DAYS_IN_PERIOD = 180


for p in persons:
    ppl += 1
    if p.vaccines:
        vax_ppl += 1

        if not p.died_at:
            vax_ppl_alive += 1

            if p.zahajeni_pojisteni < START_DATE and p.ukonceni_pojisteni > END_DATE:
                vax_ppl_not_changing_insurance += 1

                first_dose_vaccine = p.vaccines[0]
                if first_dose_vaccine.age_cohort == AgeCohort.BETWEEN_30_AND_50:
                    vax_ppl_between_30_and_50 += 1

                    if (
                        first_dose_vaccine.date >= vax_start
                        and first_dose_vaccine.date <= vax_end
                    ):
                        vax_ppl_between_30_and_50_in_vax_period += 1
                        ids.append(p.id)

                        if not p.prescriptions:
                            continue

                        for prescription in collapse_injections(p.prescriptions):
                            relative_date = (
                                prescription.date - first_dose_vaccine.date
                            ).days

                            if abs(relative_date) < DAYS_IN_PERIOD:
                                prescriptions_of_ppl_in_period += 1
                                if relative_date < 0:
                                    prescriptions_of_ppl_before_vax += 1
                                elif relative_date > 0:
                                    prescriptions_of_ppl_after_vax += 1
                                else:
                                    prescriptions_of_ppl_at_vax += 1

                        first_prescription = min(p.prescriptions, key=lambda x: x.date)
                        rel_day = (
                            first_prescription.date - first_dose_vaccine.date
                        ).days
                        if abs(rel_day) < DAYS_IN_PERIOD:
                            first_prescriptions_of_ppl_in_period += 1
                            if rel_day < 0:
                                first_prescriptions_of_ppl_before_vax += 1
                            elif rel_day > 0:
                                first_prescriptions_of_ppl_after_vax += 1
                            else:
                                first_prescriptions_of_ppl_at_vax += 1

print(f"age cohort: {AgeCohort.BETWEEN_30_AND_50}")
print(f"vax period: {vax_start} - {vax_end}")
print(f"ppl: {ppl}")
print(f"vax_ppl: {vax_ppl}")
print(f"vax_ppl_alive: {vax_ppl_alive}")
print(f"vax_ppl_not_changing_insurance: {vax_ppl_not_changing_insurance}")
print(f"vax_ppl_between_30_and_50: {vax_ppl_between_30_and_50}")
print(
    f"vax_ppl_between_30_and_50_in_vax_period: {vax_ppl_between_30_and_50_in_vax_period}"
)
print(f"prescriptions_of_ppl_in_period: {prescriptions_of_ppl_in_period}")
print(f"prescriptions_of_ppl_before_vax: {prescriptions_of_ppl_before_vax}")
print(f"prescriptions_of_ppl_after_vax: {prescriptions_of_ppl_after_vax}")
print(f"prescriptions_of_ppl_at_vax: {prescriptions_of_ppl_at_vax}")
print(f"first_prescriptions_of_ppl_in_period: {first_prescriptions_of_ppl_in_period}")
print(f"first_prescriptions_of_ppl_before_vax: {first_prescriptions_of_ppl_before_vax}")
print(f"first_prescriptions_of_ppl_after_vax: {first_prescriptions_of_ppl_after_vax}")
print(f"first_prescriptions_of_ppl_at_vax: {first_prescriptions_of_ppl_at_vax}")

age cohort: between_30_and_50
vax period: 2021-05-13 - 2021-07-13
ppl: 1570780
vax_ppl: 765972
vax_ppl_alive: 739887
vax_ppl_not_changing_insurance: 646931
vax_ppl_between_30_and_50: 215754
vax_ppl_between_30_and_50_in_vax_period: 121192
prescriptions_of_ppl_in_period: 22689
prescriptions_of_ppl_before_vax: 11467
prescriptions_of_ppl_after_vax: 11168
prescriptions_of_ppl_at_vax: 54
first_prescriptions_of_ppl_in_period: 4276
first_prescriptions_of_ppl_before_vax: 2235
first_prescriptions_of_ppl_after_vax: 2033
first_prescriptions_of_ppl_at_vax: 8
